<a href="https://colab.research.google.com/github/Ar3ee3ar/ldf_news-thai/blob/main/Topic_model_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library

In [ ]:
import pandas as pd
news = pd.read_csv('/content/drive/MyDrive/test_ML/apdf.csv')

In [ ]:
import gensim

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# dev version
# !pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip

# release version 
! pip install pythainlp

!pip install epitran
!pip install sklearn_crfsuite

In [ ]:
import pythainlp
from pythainlp import word_tokenize
!pip install git+git://github.com/rkcosmos/deepcut.git

  Cloning git://github.com/rkcosmos/deepcut.git to /tmp/pip-req-build-vsjvtqx9
  Running command git clone -q git://github.com/rkcosmos/deepcut.git /tmp/pip-req-build-vsjvtqx9
  Created wheel for deepcut: filename=deepcut-0.7.0.0-cp37-none-any.whl size=2011588 sha256=92580f45bb94d0d47ecc3b4b0c3014001a691679bd4cc41abf4b902334057da6
  Stored in directory: /tmp/pip-ephem-wheel-cache-xnbt5elx/wheels/1d/a7/f3/c9794455c7ece304440686e3e7b2ad895e1d0e7dec27d49a55
Successfully built deepcut


In [ ]:
from pythainlp.corpus import thai_stopwords
from pythainlp.util import normalize
##print(normalize("สระะน้ำ")) ##fix wrong word
list_stopwords = list(thai_stopwords())
##thai_stopwords()

In [ ]:
import numpy as np
import re ##regex
import deepcut

In [ ]:
import pandas as pd
import re
# from nltk.tokenize import word_tokenize
from pythainlp.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, LSTM, Bidirectional, Embedding, Dropout, BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.optimizers import Adam

import seaborn as sn
import matplotlib.pyplot as plt

import pickle as p
import plotly
import plotly.graph_objs as go

from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report

In [ ]:
EPOCHS = 10
BS = 32

# Select column

In [ ]:
news_th_column = news.iloc[1:,1]
news_th_array = np.array(news_th_column)
##print(news_th_array[0])

# Prepare data
### this data come with a little prepare


*   Check for null row
*   Exclude number
*   tokenize(use deepcut in pythainlp)
*   Exclude Stop words



In [ ]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13504 entries, 0 to 13503
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Column1  13504 non-null  object
 1   Column2  13504 non-null  object
dtypes: object(2)
memory usage: 211.1+ KB


In [ ]:
def cleaning(sentences):
  words = []
  temp = []
  for i in range(4000): ##test 1 data cause deepcut use so much time
    regexp_thai = re.compile(u"[^\u0E00-\u0E7Fa-zA-Z' ]|^'|'$|''")
    clean = regexp_thai.sub("",sentences[i])
    token = deepcut.tokenize(clean)
    remove_stop = [j for j in token if j not in list_stopwords]
    w = [k for k in remove_stop if k not in (' ')]
    ##w = [k for k in w in normalize(w)]
    ##print(w)
    temp.append([i.lower() for i in w])
    words.append(w)
  return words, temp

In [ ]:
cleaned_words, temp = cleaning(news_th_array)
print(len(cleaned_words))
##print(cleaned_words)

4000


# Bag of word 

### test 2 technique


*   Tokenizer from keras
*   Deepcut Tokenizer (recommend from pythainlp)



bag of word from 

https://blog.pjjop.org/sentiment-analysis-101/




In [ ]:
def create_tokenizer(words, filters = ''):
    token = Tokenizer(filters=filters)
    token.fit_on_texts(words)
    return token

In [ ]:
train_word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(train_word_tokenizer.word_index) + 1

train_word_tokenizer.word_index

{'จีน': 1,
 'พศ': 2,
 'ประเทศ': 3,
 'เรือ': 4,
 'สหรัฐฯ': 5,
 'กอง': 6,
 'ทหาร': 7,
 'ทำ': 8,
 'ปี': 9,
 'เดือน': 10,
 'ญี่ปุ่น': 11,
 'รายงาน': 12,
 'ข้อ': 13,
 'รัฐบาล': 14,
 'งาน': 15,
 'สอง': 16,
 'เกาหลีเหนือ': 17,
 'คน': 18,
 'ทัพ': 19,
 'ล้าน': 20,
 'ฝึก': 21,
 'อินเดีย': 22,
 'ทะเล': 23,
 'สร้าง': 24,
 'ภูมิภาค': 25,
 'สำหรับ': 26,
 'ระบุ': 27,
 'บิน': 28,
 'เกี่ยว': 29,
 'ฟิลิปปินส์': 30,
 'ปฏิบัติการ': 31,
 'น้ำ': 32,
 'เครื่อง': 33,
 'กฎหมาย': 34,
 'ภัย': 35,
 'ดำเนิน': 36,
 'ตัว': 37,
 'ขีปนาวุธ': 38,
 'ป้องกัน': 39,
 'มั่นคง': 40,
 'เกาหลีใต้': 41,
 'ประชุม': 42,
 'โครงการ': 43,
 'ตกลง': 44,
 'ระดับ': 45,
 'รัฐมนตรี': 46,
 'ออสเตรเลีย': 47,
 'สิทธิ': 48,
 'โลก': 49,
 'จำนวน': 50,
 'เงิน': 51,
 'โจมตี': 52,
 'พัฒนา': 53,
 'ระบบ': 54,
 'อ้าง': 55,
 'อาวุธ': 56,
 'กระทรวงกลาโหม': 57,
 'อินโดนีเซีย': 58,
 'ตะวัน': 59,
 'พื้นที่': 60,
 'ทะเลจีนใต้': 61,
 'ก่อ': 62,
 'ภาพ': 63,
 'ชาติ': 64,
 'เจ้าหน้าที่': 65,
 'สัมพันธ์': 66,
 'เมือง': 67,
 'สนับสนุน': 68,
 'ข้อมูล': 69,
 'เวลา

bag of word from
https://github.com/rkcosmos/deepcut
**use this**

In [ ]:
from deepcut import DeepcutTokenizer
def create_tokenizer_pythai(words):
  vocab = []
  tokenizer = DeepcutTokenizer(ngram_range=(1,1),max_df=1.0, min_df=0.0)
  X = tokenizer.fit_tranform(words) # 3 x 6 CSR sparse matrix
  vocab = tokenizer.vocabulary_
  return vocab

In [ ]:
#print(cleaned_words)
train_word_tokenizer_pythai = create_tokenizer_pythai(cleaned_words)
train_word_tokenizer_pythai

{'ปาปัวนิกินี': 0,
 'นิวส์': 1,
 'นางอินาดะมี': 2,
 'ไอซิส': 3,
 'กรมการผลิต': 4,
 'มหาวิทยาลัยรัฐมิชิแกน': 5,
 'ศูนย์บัญชาการกรมรบ': 6,
 'ปังกเยรี': 7,
 'ธุรกิจ': 8,
 'อุลซัน': 9,
 'อาสา': 10,
 'กลาง': 11,
 'พาดหัวข่าว': 12,
 'แอปพลิเคชัน': 13,
 'ประเทศสวีเดน': 14,
 'คองโก ซาฮาราตะวันตก': 15,
 'นางทาวน์': 16,
 'สุทธิ': 17,
 'พลรบ': 18,
 'ลีเจียน': 19,
 'อีคุระ แอนเนกซ์': 20,
 'ผูก': 21,
 'ดรเอริก ชิบูยะ': 22,
 'วานูอาตูถึง': 23,
 'ล้น': 24,
 'ห่างไกล': 25,
 'พลเพื่อ': 26,
 'ทะเบียน': 27,
 'ทักษะการ': 28,
 'นายเดวิด โคห์': 29,
 'พรรคแห่งชาตินิยม': 30,
 'ถัด': 31,
 'ซีวีเอ็น': 32,
 'โบไซ': 33,
 'ซ่งแก': 34,
 'ราบบุชมาสเตอร์': 35,
 'รัฐอานธรประเทศ': 36,
 'ล้ม': 37,
 'อาคารอพาร์ตเมนต์': 38,
 'สำนักงานตำรวจแห่งชาติเกาหลี': 39,
 'คำมั่น': 40,
 'เนชันแนล': 41,
 'ล้ำสมัย': 42,
 'เราเตอร์': 43,
 'มั่นคงและ': 44,
 'อุลจิ ฟรีดอม การ์เดียน': 45,
 'เพื่อต้าน': 46,
 'ฉันทามติ': 47,
 'สหรัฐอเมริก': 48,
 'เนรเทศ': 49,
 'เปราะ': 50,
 'ลางเลือน': 51,
 'คีออสค์': 52,
 ' จิง': 53,
 'แน่นแฟ้น': 54,
 'ประเ

In [ ]:
print(cleaned_words[0])

['ซ้าย', 'นายติโต คาร์นาเวียน', 'บัญชา', 'ตำรวจแห่งชาติ', 'อินโดนีเซีย', 'ซ้าย', 'นายโรนัลด์ เดลา โรซา', 'บัญชา', 'ตำรวจแห่งชาติ', 'ฟิลิปปินส์', 'นายคาลิด อาบู บาการ์', 'บัญชา', 'ตํารวจแห่งชาติมาเลเซีย', 'ไขว้แขน', 'ประชุม', 'มั่นคง', 'ไตรภาคี', 'เมืองปาเซย์', 'ตะวัน', 'เฉียง', 'กรุงมะนิลา', 'ประเทศฟิลิปปินส์', 'เดือน', 'มิถุนายน', 'พศ', '  ดิแอสโซซิเอทเต็ด ', 'เพรส']


In [ ]:
dictionary = gensim.corpora.Dictionary(cleaned_words)


count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break



0   ดิแอสโซซิเอทเต็ด 
1 กรุงมะนิลา
2 ซ้าย
3 ตะวัน
4 ตำรวจแห่งชาติ
5 ตํารวจแห่งชาติมาเลเซีย
6 นายคาลิด อาบู บาการ์
7 นายติโต คาร์นาเวียน
8 นายโรนัลด์ เดลา โรซา
9 บัญชา
10 ประชุม


In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in cleaned_words]
print(len(bow_corpus))
bow_corpus[3999]
bow_doc_4310 = bow_corpus[3999]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

4000
Word 13 ("กลาโหม") appears 2 time.
Word 29 ("ยุทธศาสตร์") appears 2 time.
Word 34 ("สอง") appears 1 time.
Word 67 ("ตกลง") appears 1 time.
Word 69 ("ประเทศ") appears 1 time.
Word 113 ("งาน") appears 1 time.
Word 161 ("ข้อ") appears 2 time.
Word 198 ("ทะเล") appears 1 time.
Word 243 ("รู้") appears 1 time.
Word 264 ("ทำ") appears 1 time.
Word 334 ("พื้นที่") appears 1 time.
Word 391 ("ส่งเสริม") appears 1 time.
Word 413 ("ขยาย") appears 1 time.
Word 429 ("กรอง") appears 1 time.
Word 430 ("ข้อมูลข่าว") appears 1 time.
Word 436 ("แบ่งปัน") appears 1 time.
Word 509 ("เปรียบ") appears 1 time.
Word 524 ("สัมพันธ์") appears 1 time.
Word 556 ("กรอบ") appears 1 time.
Word 618 ("บันทึก") appears 1 time.
Word 808 ("หุ้นส่วน") appears 2 time.
Word 879 ("มิติ") appears 1 time.
Word 892 ("ตระหนัก") appears 1 time.


In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

corpus_tfidf = tfidf[bow_corpus]



from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.2861565471361018),
 (1, 0.5151285619313382),
 (2, 0.14990914721657453),
 (3, 0.6242070711785629),
 (4, 0.14886945085946598),
 (5, 0.06039431532042521),
 (6, 0.13822756275656622),
 (7, 0.13788911061512185),
 (8, 0.20125641288206286),
 (9, 0.15976884491566662),
 (10, 0.18282734559600275),
 (11, 0.0874272373791335),
 (12, 0.26385665571638356)]


In [ ]:
##use bag of word
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.048*"จีน" + 0.022*"พศ" + 0.017*"เดือน" + 0.016*"รัฐบาล" + 0.015*"เกาหลีเหนือ" + 0.015*"ประเทศ" + 0.011*"ทหาร" + 0.010*"รายงาน" + 0.010*"ทำ" + 0.010*"สหรัฐฯ"
Topic: 1 
Words: 0.025*"จีน" + 0.021*"ล้าน" + 0.019*"พศ" + 0.016*"ภัย" + 0.015*"ประเทศ" + 0.012*"ก่อ" + 0.012*"ข้อ" + 0.011*"ปี" + 0.011*"คน" + 0.010*"รายงาน"
Topic: 2 
Words: 0.047*"จีน" + 0.019*"พศ" + 0.016*"ทะเล" + 0.014*"กฎหมาย" + 0.013*"สหรัฐฯ" + 0.012*"ทำ" + 0.012*"ประเทศ" + 0.011*"เกี่ยว" + 0.010*"รัฐบาล" + 0.010*"สิทธิ"
Topic: 3 
Words: 0.028*"กอง" + 0.018*"ญี่ปุ่น" + 0.016*"จีน" + 0.015*"เรือ" + 0.014*"งาน" + 0.014*"คน" + 0.014*"ขีปนาวุธ" + 0.013*"ข้อ" + 0.013*"พศ" + 0.013*"ทัพ"
Topic: 4 
Words: 0.022*"พศ" + 0.022*"กอง" + 0.021*"บิน" + 0.018*"เครื่อง" + 0.017*"ทหาร" + 0.017*"ฝึก" + 0.016*"ปี" + 0.015*"เรือ" + 0.015*"ญี่ปุ่น" + 0.015*"ทำ"
Topic: 5 
Words: 0.024*"ประเทศ" + 0.019*"ภูมิภาค" + 0.016*"เรือ" + 0.016*"ญี่ปุ่น" + 0.015*"งาน" + 0.015*"อินเดีย" + 0.012*"ทำ" + 0.011*"มั่นคง" + 0.011*"ตะวัน" + 0.009*

In [ ]:
#use tf-idf
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=4, id2word=dictionary, passes=2, workers=4)
lda_model_tfidf.save('model5.gensim')

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(n

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.013*"เรือ" + 0.011*"ฝึก" + 0.008*"กอง" + 0.008*"บิน" + 0.007*"สหรัฐฯ" + 0.007*"ญี่ปุ่น" + 0.007*"ประเทศ" + 0.007*"ทหาร" + 0.007*"เครื่อง" + 0.007*"สอง"
Topic: 1 Word: 0.008*"เกาหลีเหนือ" + 0.008*"พศ" + 0.007*"ประเทศ" + 0.007*"ออสเตรเลีย" + 0.007*"กอง" + 0.007*"อินเดีย" + 0.007*"เดือน" + 0.006*"ทหาร" + 0.006*"ปี" + 0.006*"ทัพ"
Topic: 2 Word: 0.012*"จีน" + 0.009*"สหรัฐฯ" + 0.009*"สิทธิ" + 0.009*"อ้าง" + 0.008*"เรือ" + 0.008*"ทะเลจีนใต้" + 0.007*"ทะเล" + 0.007*"ข้อ" + 0.007*"ประเทศ" + 0.007*"ทหาร"
Topic: 3 Word: 0.011*"จีน" + 0.007*"ประเทศ" + 0.007*"ญี่ปุ่น" + 0.006*"งาน" + 0.006*"ทำ" + 0.006*"เกาหลีเหนือ" + 0.006*"ล้าน" + 0.006*"เรือ" + 0.006*"คน" + 0.005*"พศ"


In [ ]:
!pip install pyLDAvis

In [ ]:
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
lda_display = gensimvis.prepare(lda, corpus_tfidf, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:479: DeprecationWarning:

This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:479: DeprecationWarning:

This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information

/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:479: DeprecationWarning:

This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information

